### Import Modules, Assign Data Path & Read into GDF

In [7]:
#Modules
import osmnx as ox 
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import matplotlib.pyplot as plt
from descartes import PolygonPatch
ox.config(log_console=True, use_cache=True)
ox.__version__
#Read Data into DataFrame
vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')
# Convert DataFrame to GeoDataFrame
vax_gdf = gpd.GeoDataFrame(vax_df, geometry=gpd.points_from_xy(vax_df['X'], vax_df['Y']))
#Reproject to UTM 17
vax_gdf.crs = 'EPSG:32617'

2022-02-23 15:07:29 Configured OSMnx 1.1.2
2022-02-23 15:07:29 HTTP response caching is on


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_58175/4099419668.py:16: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  vax_df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/COVID_VaccineClinics/Covid-19_Vaccination_Provider_Locations_in_the_United_States.csv')


### Limit to Detroit Metro Area (Which we are defining as Wayne, Oakland, and Macomb County)

In [8]:
#limit by state to Michigan
vax_MI = vax_gdf[vax_gdf['State']== 'MI'] 
#Limit By County to Wayne, Oakland, and Macomb
vax_wayne = vax_MI[vax_MI['county']== 'Wayne']
vax_oakland = vax_MI[vax_MI['county']== 'Oakland'] 
vax_macomb = vax_MI[vax_MI['county']== 'Macomb']
#Agglomerate to one GDF of Detroit Metro
vax_detr_metro = vax_wayne 
vax_detr_metro = vax_detr_metro.append(vax_oakland)
vax_detr_metro = vax_detr_metro.append(vax_macomb)

/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_58175/2395504573.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_oakland)
/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_58175/2395504573.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vax_detr_metro = vax_detr_metro.append(vax_macomb)


In [21]:
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index('id')

    edge_lines = []
    for n_fr, n_to in subgraph.edges():
        f = nodes_gdf.loc[n_fr].geometry
        t = nodes_gdf.loc[n_to].geometry
        edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
        edge_lines.append(edge_lookup)

    n = nodes_gdf.buffer(node_buff).geometry
    e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
    all_gs = list(n) + list(e)
    new_iso = gpd.GeoSeries(all_gs).unary_union
        
    # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
    if infill:
        new_iso = Polygon(new_iso.exterior)
    isochrone_polys.append(new_iso)
    return isochrone_polys

In [22]:
def isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute):
    network_type = network_type
    trip_time = trip_time
    travel_speed = travel_speed
    meters_per_minute = meters_per_minute
    # download the street network
    G = ox.graph_from_point(place, network_type=network_type)
    # find the centermost node and then project the graph to UTM
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    x, y = gdf_nodes['geometry'].unary_union.centroid.xy
    center_node = ox.get_nearest_node(G, (y[0], x[0]))
    G = ox.project_graph(G)
    for u, v, k, data in G.edges(data=True, keys=True):
        data['time'] = data['length'] / meters_per_minute
    isochrone_polys = []
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)
    isochrone_polys = make_iso_polys(G, center_node, edge_buff=25, node_buff=0, infill=True)
    gdf_iso_poly = gpd.GeoDataFrame(isochrone_polys)
    return gdf_iso_poly

In [23]:
#Set Variables
network_type = 'walk'
trip_time = 20 #in minutes
travel_speed = 4.5 #walking speed in km/hour
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
detroit_polygons = gpd.GeoDataFrame() 

In [25]:
for i in range(len(vax_detr_metro)):
    place = vax_detr_metro['Y'].iloc[i],vax_detr_metro['X'].iloc[i]
    gdf_iso_poly=isochrone_shape_generator(place, network_type, trip_time, travel_speed, meters_per_minute)
    detroit_polygons = detroit_polygons.append(gdf_iso_poly)

2022-02-23 15:15:19 Created bbox 1000 m from (42.3963769002206, -83.4806078996323): 42.40537010357553,42.387383696865676,-83.46843020434454,-83.49278559492006
2022-02-23 15:15:19 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:15:19 Projected GeoDataFrame to epsg:4326
2022-02-23 15:15:19 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:15:19 Projected GeoDataFrame to epsg:4326
2022-02-23 15:15:19 Requesting data within polygon from API in 1 request(s)
2022-02-23 15:15:20 Retrieved response from cache file "cache/57002b79dcee13f1efd36a0e9f1512a90a7b4c00.json"
2022-02-23 15:15:20 Got all network data within polygon from API in 1 request(s)
2022-02-23 15:15:20 Creating graph from downloaded OSM data...
2022-02-23 15:15:20 Created graph with 4232 nodes and 9136 edges
2022-02-23 15:15:20 Added length attributes to graph edges
2022-02-23 15:15:20 Identifying all no

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-23 15:15:27 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:15:27 Created edges GeoDataFrame from graph
2022-02-23 15:15:28 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:15:28 Created graph from node/edge GeoDataFrames
2022-02-23 15:15:28 Projected graph with 469 nodes and 1300 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_58175/3459140347.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)


2022-02-23 15:15:34 Created bbox 1000 m from (42.3903256997602, -82.9147948003165): 42.39931890311512,42.38133249640527,-82.90261827909424,-82.92697132153876
2022-02-23 15:15:34 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:15:34 Projected GeoDataFrame to epsg:4326
2022-02-23 15:15:34 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:15:34 Projected GeoDataFrame to epsg:4326
2022-02-23 15:15:34 Requesting data within polygon from API in 1 request(s)
2022-02-23 15:15:34 Retrieved response from cache file "cache/76c1edeab297279db46d8b790aa123a27fd7a24b.json"
2022-02-23 15:15:34 Got all network data within polygon from API in 1 request(s)
2022-02-23 15:15:34 Creating graph from downloaded OSM data...
2022-02-23 15:15:34 Created graph with 2833 nodes and 6666 edges
2022-02-23 15:15:34 Added length attributes to graph edges
2022-02-23 15:15:35 Identifying all nod

...

/Users/zachary/opt/anaconda3/envs/final_ML_LR/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


2022-02-23 15:22:42 Created nodes GeoDataFrame from graph
2022-02-23 15:22:42 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:22:43 Created edges GeoDataFrame from graph
2022-02-23 15:22:44 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:22:44 Created graph from node/edge GeoDataFrames
2022-02-23 15:22:44 Projected graph with 568 nodes and 1834 edges


/var/folders/8v/6dq3dnld7rx4qv5wr7vx0w5m0000gn/T/ipykernel_58175/3459140347.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  detroit_polygons = detroit_polygons.append(gdf_iso_poly)


2022-02-23 15:22:53 Created bbox 1000 m from (42.2579800997193, -83.2055310996776): 42.266973303074224,42.24898689636437,-83.19338016605191,-83.21768203330329
2022-02-23 15:22:53 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:22:53 Projected GeoDataFrame to epsg:4326
2022-02-23 15:22:53 Projected GeoDataFrame to +proj=utm +zone=17 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs
2022-02-23 15:22:53 Projected GeoDataFrame to epsg:4326
2022-02-23 15:22:53 Requesting data within polygon from API in 1 request(s)
2022-02-23 15:22:53 Resolved overpass-api.de to 178.63.48.217
2022-02-23 15:22:54 Pausing 0 seconds before making HTTP POST request
2022-02-23 15:22:54 Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A180%5D%3B%28way%5B%22highway%22%5D%5B%22area%22%21~%22yes%22%5D%5B%22access%22%21~%22private%22%5D%5B%22highway%22%21~%22abandoned%7Cbus_guideway%7Cconstruction%7Ccycleway%7Cmotor%7Cpl

KeyboardInterrupt: 

In [27]:
detroit_polygons.head(20)

,0
0,"POLYGON ((294860.4613026406 4695961.132099903,..."
0,"POLYGON ((341502.0415138075 4694202.765720206,..."
0,POLYGON ((318063.13724238676 4699273.635123259...
0,"POLYGON ((323800.8485959251 4688184.412598591,..."
0,"POLYGON ((321421.9978007265 4699316.824453924,..."
0,"POLYGON ((312706.9652684526 4670811.079845155,..."
0,"POLYGON ((300634.9053449211 4686431.964118528,..."
0,"POLYGON ((294944.9174387326 4699953.680981675,..."
0,"POLYGON ((313353.0704268181 4680012.425938412,..."
0,"POLYGON ((302310.1485699231 4684977.210632092,..."
